In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm


from sklearn.metrics import f1_score

import zipfile
import os



In [ ]:
# mount the drive where your dataset is available
from google.colab import drive
drive.mount('/content/drive')
filepath='/content/drive/MyDrive/datasets/multimodal_product_classification/' # add your own path. Where to save the dataset


Mounted at /content/drive


In [ ]:
if not os.path.exists('datasets'):
  os.makedirs('datasets')
  with zipfile.ZipFile(filepath+'images.zip', 'r') as zip_ref:
      zip_ref.extractall('datasets')